In [1]:
# !pip install pandas_gbq
# !pip install pydata_google_auth
# !pip install google-cloud-bigquery-storage

In [15]:
import pandas as pd
import numpy as np

from google.cloud import bigquery
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(r'C:\Users\kevin.puika\OneDrive - Bank Jasa Jakarta\Desktop\KRP-CKPN\joey-bi-ss-risk-fraud-project-36e37a4eccd7 1.json')
project_id = "joey-bi-ss-risk-fraud-project"

client = bigquery.Client(credentials= credentials,project=project_id)


In [3]:
## Where 2 mths before, for FPD14
sql = '''
    select *
    from joey-bi-ss-risk-fraud-project.credit.feat_eng_1_2025_09_30
    where disb_dt <= '2025-07-31'
'''
feat_eng1 = client.query(sql).to_dataframe()
feat_eng1.head()

,t24_customer_id,limit_id,deviation_level,limit_amount,request_ip_address,request_location,request_gps,req_gps_latitude,req_gps_longitude,device_id,...,device_phone_length,demog_age_grp,demog_company_grp,Demog_Industry_grp,Demog_occu_grp,Device_email_domain_grp,clik_bur_score_grp_2,bps_score_grp_2,dpd_os_14_acct,dpd_os_30_acct
0,100000050,107713,A,30000000.000000000,140.213.18.150,Indonesia,"-6.1159159,106.7592634",-6.1159159,-6.1159159,2e8409651b7ab320,...,13,4. Age >= 38,01. Company: Bank,"02. Mining, Bank",1. Karyawan Swasta,1. Gmail,03. 380 - 519,00. null,0,0
1,100000050,107713,A,30000000.000000000,140.213.18.150,Indonesia,"-6.1159159,106.7592634",-6.1159159,-6.1159159,2e8409651b7ab320,...,13,4. Age >= 38,01. Company: Bank,"02. Mining, Bank",1. Karyawan Swasta,1. Gmail,03. 380 - 519,00. null,0,0
2,102264080,106450,None,800000.000000000,182.253.14.104,Indonesia,"-6.2333244,106.6477881",-6.2333244,-6.2333244,e87af9b6271de543,...,12,3. Age is 23-38,10. Company: PT.,09. Manufacture,1. Karyawan Swasta,1. Gmail,04. 520 - 545,00. null,0,0
3,100341952,2151,None,500000.000000000,114.79.7.49,China,"23.135241056971577,113.32972308107493",23.135241056971577,23.135241056971577,7368cad3ced421b2,...,13,2. Age is 22,99. Others Company,"10. Hotel, Travel, & Developer",1. Karyawan Swasta,1. Gmail,04. 520 - 545,05. 594 - 627,0,0
4,101304014,68075,None,13000000.000000000,120.188.67.158,Indonesia,"-6.1879062,106.7380936",-6.1879062,-6.1879062,f347f4c4ccadf62b,...,13,3. Age is 23-38,10. Company: PT.,13. Others,1. Karyawan Swasta,1. Gmail,04. 520 - 545,00. null,0,0


In [4]:
feat_eng1.to_csv("Full_data_mdl.csv", index = False)

In [5]:
## Number of Samples
feat_eng1[['arrangement_id']].count()

arrangement_id    2490090
dtype: int64

In [16]:
## Number of Bad
feat_eng1[['dpd_os_30_acct']][feat_eng1['dpd_os_30_acct'] > 0 ].count()

dpd_os_30_acct    91912
dtype: int64

In [66]:
## make list of features

feat_list = list(feat_eng1.columns.values)
remove = {      't24_customer_id','limit_id','deviation_level','limit_amount','request_ip_address','request_location','request_gps','req_gps_latitude',
'req_gps_longitude','device_id','status','apply_hr','apply_date','nik','brand','model','os_version','arrangement_id',
'disb_amt','tenor','product_code','disb_dt','disb_mth','disb_hr','phone_number','date_of_birth','company','emergency_contact',
'emergency_contact_relationship','emergency_contact_number','residential_status','urban_village','postal_code',
'office_phone','office_address','dpd_os_30','dpd_os_14','prev_liab_balance','liab_balance','pocket_balance','sakubooster_balance',
'mainpocket_balance','bsp_result','os_client','bps_score_raw','bps_score','device_IP_last_2_layer','device_IP_last_layer','device_model_brand',
'device_IP_third_layer','declared_income','savingpocket_balance','device_IP_fourth_layer','spendingpocket_balance','payroll_balance','inflow_amount',
'clik_bur_score','device_IP_last_4_layer','trfr_out_amount','device_IP_last_3_layer','outflow_amount','trfr_in_amount','pocket_number',
'device_os_version','device_IP_second_layer','intrabank_amount','place_of_birth',
'length_of_stay','residential_type','residential_address','district','residential_neighborhood','source_of_funds','dpd_os_14_acct','dpd_os_30_acct',


# good one:          
'mob_at','ever_busposito','inflow_freq','outflow_freq','active_trx_customer_payment_team_P7D','Actv_tabungmatic_rate_P14D',
'Actv_qris_P31_60D', 'Actv_tabungmatic_rate_P7D', 'Actv_qris_rate_P30D','Actv_trfout_rate_P30D','ever_qris',''
}

feat_list = list(filter(lambda x: x not in remove, feat_list))
# feat_list
# 

In [67]:
## Features grouping
n = len(feat_list)
feat_eng1_grp = pd.DataFrame()

for i in range(n):
    feat_eng1_ph = feat_eng1[[feat_list[i],'dpd_os_14_acct']].groupby(feat_list[i]).agg(['count','sum'])
    feat_eng1_ph = feat_eng1_ph.reset_index()
    feat_eng1_ph.columns = feat_eng1_ph.columns.droplevel(level=0)
    feat_eng1_ph = feat_eng1_ph.rename(columns= {'' : 'details', 'count' : 'num_acc', 'sum' : 'num_bad'})
    feat_eng1_ph.insert(0, 'Features', feat_list[i])

    if feat_eng1_grp.empty:
        feat_eng1_grp = feat_eng1_ph
    else:
        feat_eng1_grp = pd.concat([feat_eng1_grp, feat_eng1_ph], ignore_index=True)


In [68]:
## Calc
feat_eng1_grp['Event'] = feat_eng1_grp['num_bad'] +0.5
feat_eng1_grp['Non-Event'] = feat_eng1_grp['num_acc'] - feat_eng1_grp['num_bad'] +0.5
feat_eng1_grp['Perc_NE'] = feat_eng1_grp['Non-Event'] / feat_eng1_grp.groupby('Features')['Non-Event'].transform('sum')
feat_eng1_grp['Perc_E'] = feat_eng1_grp['Event'] / feat_eng1_grp.groupby('Features')['Event'].transform('sum')
feat_eng1_grp['WOE'] = np.log(feat_eng1_grp['Perc_NE']/feat_eng1_grp['Perc_E'])
feat_eng1_grp['IV'] = (feat_eng1_grp['Perc_NE']-feat_eng1_grp['Perc_E']) * feat_eng1_grp['WOE']


In [69]:
## Feats IV
feat_eng1_grp.groupby('Features').agg({'IV':'sum'}).sort_values(by = 'IV', ascending=False)

,IV
Features,
Actv_reward_P14D,0.992231
Active_rate_P7D,0.989325
Actv_trfout_rate_P14D,0.986328
Active_trx_rate_P7D,0.983923
active_trx_customer_payment_team_P30D,0.949963
...,...
channel,0.000724
aai_verify_type,0.00006
refund_amount,0.0


In [40]:
## Drop to excel:

feat_eng1_grp.to_excel("feats_IV_2025_09.xlsx",    sheet_name='Sheet_name_1') 